### Pre-training
Train on unstructured data

In [1]:
!pip install -U bitsandbytes accelerate transformers datasets peft mlflow safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128

In [2]:
from datasets import Dataset
from transformers import Trainer

DATASET_PATH = '/content/runescape_corpus.txt'

class TrainingUtils:
    def __init__(self):
        pass

    def load_dataset(self, path):
        with open(path, mode='r', encoding='utf-8') as f:
            data_list = f.readlines()
            data_list = [line.strip() for line in data_list]
        data_dict = {"text": data_list}
        dataset = Dataset.from_dict(data_dict)
        return dataset


class LlamaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        try:
            labels = inputs.get("input_ids")
            outputs = model(**inputs)
            # torch.cuda.synchronize()
            logits = outputs.get("logits")

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # torch.cuda.synchronize()

            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            # torch.cuda.synchronize()
            return (loss, outputs) if return_outputs else loss
        except Exception as e:
            print(e)

utils = TrainingUtils()
dataset = utils.load_dataset(DATASET_PATH)

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from accelerate import Accelerator
from torch import nn
from google.colab import userdata


MODEL_PATH = 'meta-llama/Llama-3.2-1B'
access_token = userdata.get('HF_TOKEN')

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, token=access_token)
EOS_TOKEN = tokenizer.eos_token
tokenizer.pad_token = EOS_TOKEN

def add_eos_token_func(text):
    texts = text['text']
    texts_formatted =  [text + EOS_TOKEN for text in texts]
    return {"text": texts_formatted}
dataset_formatted = dataset.map(add_eos_token_func, batched=True)
dataset_tokenized = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding=True, return_tensors="pt"), batched=True)
dataset_tokenized.set_format(type='torch', columns = ['input_ids', 'attention_mask'])

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, token=access_token)
model = prepare_model_for_kbit_training(model)
model.resize_token_embeddings(len(tokenizer))

lora_config = LoraConfig(
    inference_mode=False,
    r=4,
    lora_alpha=4,
    # for fine-tuning
    # target_modules=["q_proj", "k_proj", "v_proj"],
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head",],
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_config)
accelerator = Accelerator()
model = accelerator.prepare(model)
model = model.to(accelerator.device)

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_strategy="no",
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    remove_unused_columns=False,
    report_to="none",
    fp16=True,
    gradient_accumulation_steps=2,
)

trainer = LlamaTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenized,
)

trainer.args._n_gpu = 1
print(f"TRAINING WITH {torch.cuda.device_count()} GPUs...")
print(trainer.train())
OUTPUT_DIR = "content/trained_model_safetensors"

model.save_pretrained(OUTPUT_DIR, safe_serialization=True)
tokenizer.save_pretrained(OUTPUT_DIR)


Map:   0%|          | 0/179 [00:00<?, ? examples/s]

Map:   0%|          | 0/179 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:498: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


TRAINING WITH 1 GPUs...


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=267, training_loss=0.7560799130786224, metrics={'train_runtime': 180.8869, 'train_samples_per_second': 2.969, 'train_steps_per_second': 1.476, 'total_flos': 1023617622491136.0, 'train_loss': 0.7560799130786224, 'epoch': 2.983240223463687})


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('content/trained_model_safetensors/tokenizer_config.json',
 'content/trained_model_safetensors/special_tokens_map.json',
 'content/trained_model_safetensors/tokenizer.json')